In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read data
df = pd.read_csv('data-sets/air_passengers.csv')

# Create lagged features
for i in range(1, 3):
    df[f'lag_{i}'] = df['Passengers'].shift(i)

# Drop NA values
df.dropna(inplace=True)

# Define inputs and target
X = df[['lag_1', 'lag_2']].values
y = df['Passengers'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)  # 0.2 x 0.8 = 0.16


In [34]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import torch
from torch import nn

class LSTM(pl.LightningModule):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=2, num_layers=1, lr=0.1, weight_decay=0.01):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.lr = lr
        self.weight_decay = weight_decay
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.loss_func = nn.MSELoss()

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq) ,1, -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_func(y_hat, y)
        return loss


In [35]:

# Parameters for grid search
learning_rates = [0.01, 0.1]
epochs = [50, 100]
hidden_sizes = [50, 100]
layers = [1, 2]
weight_decays = [1e-3, 1e-2]

from torch.utils.data import DataLoader, TensorDataset
# Preparing data loaders
train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
val_dataset = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)



# Iterate over hyperparameters
for lr in learning_rates:
    for epoch in epochs:
        for hidden_size in hidden_sizes:
            for layer in layers:
                for weight_decay in weight_decays:
                    model = LSTM(hidden_layer_size=hidden_size, num_layers=layer, lr=lr, weight_decay=weight_decay)
                    trainer = Trainer(max_epochs=epoch)
                    trainer.fit(model, train_loader, val_loader)


AttributeError: module 'pytorch_lightning' has no attribute 'LightningModule'